# Module 1

## ETL for MELI Data Challenge 2021

In [2]:
!pip install tqdm
!pip install jsonlines

import pandas as pd
import json
from tqdm import tqdm
import csv
import matplotlib.pyplot as plt
import multiprocessing
from itertools import chain, islice
from datetime import timedelta
import jsonlines
import seaborn as sns
from pathlib import Path

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 1. Fetching Data

#### Loading train and test datasets

In [4]:
df = pd.read_parquet('/content/gdrive/MyDrive/Data/train_data.parquet')
data_test = pd.read_csv('/content/gdrive/MyDrive/Data/test_data.csv')

In [5]:
df.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0


In [6]:
data_test.head()

,sku,target_stock
0,464801,3
1,645793,4
2,99516,8
3,538100,8
4,557191,10


Load extra item data

In [7]:
## auxiliary function to read jsonlines files

def load_jsonlines(filename):
  rv = []
  for obj in tqdm(jsonlines.open(filename)):
    rv.append(obj)
  return rv

In [8]:
item_metadata = load_jsonlines('/content/items_static_metadata_full.jl')

660916it [00:06, 107095.62it/s]


Convert to a df and use sku as the index

In [9]:
df_metadata = pd.DataFrame(item_metadata)
df_metadata.index = df_metadata.sku
df_metadata.drop(columns=['sku'], inplace = True)

In [10]:
df_metadata.head()

,item_domain_id,item_id,item_title,site_id,product_id,product_family_id
sku,,,,,,
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,None,None


Hydrate initial datasets with the extra data

In [11]:
df_sample = df.sample(frac=0.1, random_state=1)

In [13]:
df_sample = df_sample.join(df_metadata, on='sku', how='left')

In [16]:
data_test = data_test.join(df_metadata, on='sku', how='left')

In [14]:
df_sample.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,item_id,item_title,site_id,product_id,product_family_id
1005365,152848,2021-03-24,1,167.00,REA,premium,fulfillment,free_shipping,1041.411833,MLB-DRESSES,3892,Vestido Midi Mulher Virtuosa,MLB,None,None
562310,431324,2021-02-09,27,28.49,REA,premium,fulfillment,paid_shipping,1440.000000,MLB-CELLPHONE_PARTS,56302,Cola Preta T7000 15ml Para Celular Reparo Touc...,MLB,None,None
1384579,525309,2021-03-04,0,37.91,REA,premium,fulfillment,paid_shipping,1440.000000,MLB-VEHICLE_STICKERS,494687,Kit Adesivos Dodge Dakota Sport 3.9 V6 Em Prat...,MLB,None,None
33950609,492760,2021-02-18,0,169.15,MEX,premium,drop_off,paid_shipping,1440.000000,MLM-CELLPHONE_COVERS,190467,Fundas Nintendo Switch Para iPhone 11 iPhone 1...,MLM,None,None
9537449,223511,2021-02-17,0,24.90,REA,premium,fulfillment,paid_shipping,0.000000,MLB-KITCHEN_SUPPLIES,467072,Cesto Cozimento A Vapor Inox Legumes Cozinha P...,MLB,None,None


### Now we'll save the data sample to a csv in order to have it available to work in the future

In [15]:
df_sample.to_csv('/content/gdrive/MyDrive/Data/df_sample.csv')